In [1]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import os
import random 

### Preliminary tests/book-keeping

In [2]:
class_names = ['10c', '10d', '10h', '10s', '2c', '2d', '2h', '2s', '3c', '3d', '3h', '3s', '4c', '4d', '4h', '4s', '5c', '5d', '5h', '5s', '6c', '6d', '6h', '6s', '7c', '7d', '7h', '7s', '8c', '8d', '8h', '8s', '9c', '9d', '9h', '9s', 'Ac', 'Ad', 'Ah', 'As', 'Jc', 'Jd', 'Jh', 'Js', 'Kc', 'Kd', 'Kh', 'Ks', 'Qc', 'Qd', 'Qh', 'Qs']

In [3]:
# Useful functions
def IMGwBB(class_names, image_dir, label_dir, subplot_size):
    image_files = os.listdir(image_dir)[:9]
    label_files = []

    for i in range(len(image_files)):
        label_files.append(image_files[i].replace('.jpg', '.txt'))

    
    sp_r, sp_c = subplot_size
    fig, axes = plt.subplots(sp_r, sp_c,figsize=(12,12))
    axes = axes.ravel()
        
    for i in range(len(image_files)):     
        img_path = os.path.join(image_dir, image_files[i])
        label_path = os.path.join(label_dir, label_files[i])
        
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, _ = img.shape
    
        with open(label_path, "r") as f:
            labels = f.readlines()
    
        for label in labels:
            cid, xc, yc, w, h = map(float, label.split())
            x1 = int((xc - w / 2) * width)
            y1 = int((yc - h / 2) * height)
            x2 = int((xc + w / 2) * width)
            y2 = int((yc + h / 2) * height)
    
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 0), 2)
            cv2.putText(img, str(class_names[int(cid)]), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6,(255, 255, 0), 2)
        
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(f'Image: {i + 1}')
    
    plt.tight_layout()
    plt.show()
  

In [4]:
def label_stats(class_names, label_dir):
    label_files = os.listdir(label_dir)
    nc = len(class_names) # number of classes
    class_counts = [0] * nc
    
    for i in range(len(label_files)):     
        label_path = os.path.join(label_dir, label_files[i])
        # img_path = os.path.join('./yolo_data/train/images/', label_files[0].replace('.txt', '.jpg'))

        # img = cv2.imread(img_path)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # height, width, _ = img.shape
        # plt.imshow(img)
        
        with open(label_path, "r") as f:
            labels = f.readlines()

        for label in labels: 
            cid, xc, yc, w, h = map(float, label.split())
            cid = int(cid)
            class_counts[cid] += 1

    max_id = np.argmax(class_counts)
    max_count = class_counts[max_id]
    max_class = class_names[max_id]

    min_id = np.argmin(class_counts)
    min_count = class_counts[min_id]
    min_class = class_names[min_id]

    # based on number (ignore suits):
    class_counts_ns = [0] * (nc//4)
    for i in range(0, nc, 4):
        class_counts_ns[i//4] = np.sum(class_counts[i:i+4])
    
    
    print(f'Class count list: {class_counts}')
    print(f'Class count list (no suits): {class_counts_ns}')
    
    max_id_ns = np.argmax(class_counts_ns)
    max_count_ns = class_counts_ns[max_id_ns]
    max_class_ns = class_names[max_id_ns * 4][:-1]

    min_id_ns = np.argmin(class_counts_ns)
    min_count_ns = class_counts_ns[min_id_ns]
    min_class_ns = class_names[min_id_ns * 4][:-1]

    total_cards = np.sum(class_counts)
    baseline_error = (total_cards - max_count) * 100 / total_cards
    baseline_error_ns  = (total_cards - max_count_ns) * 100 / total_cards
    print(f'total cards: {total_cards}')
    print(f'max card: {max_class}\t count: {max_count}')
    print(f'min card: {min_class}\t count: {min_count}')

    print(f'max card (no suits): {max_class_ns}\t count: {max_count_ns}')
    print(f'min card (no suits): {min_class_ns}\t count: {min_count_ns}')

    print(f'baseline error: {baseline_error}')
    print(f'baseline error (no suits): {baseline_error_ns}')

    return class_counts, class_counts_ns         

In [5]:
train_img_dir = './datasets/train/images/'
train_label_dir = './datasets/train//labels/'

val_img_dir = './datasets/valid/images/'
val_label_dir = './datasets/valid/labels/'

test_img_dir = './datasets/test/images/'
test_label_dir = './datasets/test/labels/'

print(len(os.listdir(train_img_dir)))
print(len(os.listdir(val_img_dir)))
print(len(os.listdir(test_img_dir)))

# cc, cc_ns = label_stats(class_names, label_dir)
# IMGwBB(class_names, image_dir, label_dir, (3,3))
print('\nTRAIN DATA STATS\n----------')
cc, cc_ns = label_stats(class_names, train_label_dir)
print('\nVALIDATION DATA STATS\n----------')
cc, cc_ns = label_stats(class_names, val_label_dir)
print('\nTEST DATA STATS\n----------')
cc, cc_ns = label_stats(class_names, test_label_dir)


14000
4000
2000

TRAIN DATA STATS
----------
Class count list: [986, 999, 1005, 1032, 1062, 1071, 1046, 955, 1034, 973, 1098, 1066, 985, 997, 1117, 1005, 942, 1001, 1039, 1004, 997, 1039, 993, 1011, 994, 1085, 978, 978, 1012, 1171, 989, 998, 950, 980, 1049, 1006, 1062, 1015, 1028, 1042, 1022, 1075, 971, 1013, 1014, 1014, 1034, 1037, 938, 1050, 1046, 995]
Class count list (no suits): [4022, 4134, 4171, 4104, 3986, 4040, 4035, 4170, 3985, 4147, 4081, 4099, 4029]
total cards: 53003
max card: 8d	 count: 1171
min card: Qc	 count: 938
max card (no suits): 3	 count: 4171
min card (no suits): 9	 count: 3985
baseline error: 97.790691092957
baseline error (no suits): 92.13063411505009

VALIDATION DATA STATS
----------
Class count list: [282, 290, 322, 250, 295, 259, 294, 299, 308, 250, 313, 358, 276, 282, 312, 295, 290, 283, 253, 281, 284, 286, 329, 289, 304, 311, 303, 304, 285, 261, 255, 339, 309, 275, 272, 263, 288, 294, 283, 322, 265, 285, 278, 282, 307, 304, 297, 264, 300, 317, 295, 317]
Cla

### YOLOv5 training

In [6]:
model = YOLO('yolov5nu.pt')

In [7]:
quick_model = model.train(data = 'data.yaml', 
                          imgsz = 416,
                          epochs=30,
                          batch=-1, 
                          device="mps",
                          optimizer = 'auto', 
                          plots=True, 
                          verbose=True,
                          project='./models',
                          name='good_model'
                         )

Ultralytics 8.3.70 🚀 Python-3.12.8 torch-2.6.0 MPS (Apple M4)
engine/trainer: task=detect, mode=train, model=yolov5nu.pt, data=data.yaml, epochs=30, time=None, patience=100, batch=-1, imgsz=416, save=True, save_period=-1, cache=False, device=mps, workers=8, project=./models, name=good_model2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, for

train: Scanning /Users/neosim/Developer/Undergraduate/Y4_Senior/CSSE-463/dataset

AutoBatch: Computing optimal batch size for imgsz=416 at 60.0% CUDA memory utilization.
AutoBatch:  ⚠️ intended for CUDA devices, using default batch-size 16



train: Scanning /Users/neosim/Developer/Undergraduate/Y4_Senior/CSSE-463/dataset
val: Scanning /Users/neosim/Developer/Undergraduate/Y4_Senior/CSSE-463/datasets/


Plotting labels to models/good_model2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000179, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to models/good_model2
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4000      15159      0.671      0.745      0.764       0.66

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/30      17.2G     0.7477      1.402     0.8264         62        416: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       4000      15159      0.735      0.802      0.832      0.716

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       8/30      18.8G     0.7266      1.281     0.8234         66        416: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


RuntimeError: MPS backend out of memory (MPS allocated: 2.02 GB, other allocations: 16.04 GB, max allowed: 18.13 GB). Tried to allocate 98.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).